In [19]:
import pandas as pd
import json
import itertools
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [20]:
# Authorization Code Flow
# I need the user-top-read authentication to get my users top tracks 
SCOPE = 'user-top-read'
CLIENT_ID = "03047be420ab4ef482e5312d78a720f0"
CLIENT_SECRET = "404f1e8783794d56b2c7fa475c20933e"
REDIRECT_URI = "http://localhost:3000"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=SCOPE, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI))

In [57]:
# Get my top tracks
# we do all the ranges ("long_term" "medium_term", "short_term") to get most of the tracks since we can only get 50 tracks at a time.
# I also exploit a bug where offsetting 49 can get you 49 more tracks, per range!

tt_long_first = sp.current_user_top_tracks(limit=50, offset=0, time_range='long_term')
tt_long_second = sp.current_user_top_tracks(limit=50, offset=49, time_range='long_term')

tt_medium_first = sp.current_user_top_tracks(limit=50, offset=0, time_range='medium_term')
tt_medium_second = sp.current_user_top_tracks(limit=50, offset=49, time_range='medium_term')

tt_short_first = sp.current_user_top_tracks(limit=50, offset=0, time_range='short_term')
tt_short_second = sp.current_user_top_tracks(limit=50, offset=49, time_range='short_term')

In [ ]:
#check that all responses have 50 items in them
print(len(tt_long_first["items"]))
print(len(tt_long_second["items"]))
print(len(tt_medium_first["items"]))
print(len(tt_medium_second["items"]))
print(len(tt_short_first["items"]))
print(len(tt_short_second["items"]))

In [113]:
# filter out and add only track name, id and range to one big dataframe.
# associate range with top_track responses
tt_combined = [
    {"range": "long_term", 
     "val": [tt_long_first, tt_long_second]},
    {"range": "medium_term", 
     "val": [tt_medium_first, tt_medium_second]},
    {"range": "short_term", 
     "val": [tt_short_first, tt_short_second]}
]

# create a result dataframe
df_toptracks = pd.DataFrame()

# go throught all top tracks, filter and add to one one big bf
for item in tt_combined:
    curr_range = item["range"]

    for tt in item["val"]:    
        curr_tt = tt["items"]
        # add to dataframe
        df_tt = pd.DataFrame(curr_tt)

        # filter away everything exept track name and track id
        df_tt_filt = df_tt.filter(["id", "name"])

        # add coulumn with the range
        df_tt_filt["range"] = curr_range
        
        # add to one big dataframe
        df_toptracks = pd.concat([df_toptracks, df_tt_filt], ignore_index=True)

        
# print stuff
print("rows in final df:", len(df_toptracks))
df_toptracks

rows in final df: 300


,id,name,range
0,2E638Wy85ULliAXTQEFET0,The Border,long_term
1,6ZHaA7gE78xjsJ9MZCBw8x,Awake,long_term
2,6kWzCCAghrDeVFZSKgYVGL,"Mass in B Minor, BWV 232: Kyrie: Kyrie eleison...",long_term
3,0uoHVRsDwVHtzYLMvRkxtC,Automatic,long_term
4,2Ghp894n1laIf2w98VeAOJ,How Do I Make You Love Me?,long_term
...,...,...,...
295,3kI3vRDwTbYs9O8svalUWu,No Choice,short_term
296,407eqFGqQc40KPf43YrQ44,Asche zu Asche,short_term
297,4L9UGREMQBfYLmGwlACgTV,Moskau,short_term
298,4T1iiabe7G0UjWQJCY6NE2,Videotape,short_term


In [ ]:
# BUT, make sure that they are all in the correct order!! so most popular on top, and decending downwards.
# Make this algorithm,
# then, print out to csv. 